# Running Pyomo on Google Colab

Google Colab offers a web-based computational platform for Python and Jupyter notebooks, which can be accessed through a simple web browser. By using this service, users can avoid the burden of setting up and managing a complicated computing environment on their personal computers.

This notebook provides instructions on how to install the pyomo package and open-source solvers on Google Colab. The solvers that will be installed are:

| Solver | Description |
| :-- | :-- |
| GLPK | MILP |
| [COIN-OR CLP](https://github.com/coin-or/Clp) | LP
| [COIN-OR CBC](https://github.com/coin-or/Cbc) | MILP |
| [COIN-OR Ipopt](https://github.com/coin-or/Ipopt) | NLP | |
| [COIN-OR Bonmin](https://github.com/coin-or/Bonmin) | MINLP |
| [COIN-OR Couenne](https://github.com/coin-or/Couenne) | Global MINLP |

Additionally, the notebook tests each solver using a small example problem.

**Note: This notebook has been updated to use pyomo and solvers associated with the [IDAES project](https://idaes.org/). As of this writing (Feb, 2023), IDAES is undergoing an update to Version 2.0. This notebook will be updated as needed to stay current with IDAES.**

## Installing Pyomo and Solvers

An installation of pyomo and solvers needs to be done once at the start of each Colab session. The following cell tests if it is being run on Google Colab and, if so, installs Pyomo and solvers from the IDAES proejct. The [cell magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-capture) `%%capture` captures the lengthy output from the installation scripts. See the documentation for [installing the IDAES PSE framework](https://idaes-pse.readthedocs.io/en/latest/tutorials/getting_started/index.html) for additional detail.

In [1]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions
    !apt-get install -y -qq glpk-utils  
    os.environ['PATH'] += ':/usr/bin:/root/.idaes/bin'

## Test Model

The installation of pyomo can be verified by entering a simple model. We'll use the model again in subsequent cells to demonstrate the installation and execution of various solvers.

In [2]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(expr = 40*model.x + 30*model.y, sense=maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

model.pprint()

def display_solution(model):

    # display solution
    print('\nProfit = ', model.profit())

    print('\nDecision Variables')
    print('x = ', model.x.value)
    print('y = ', model.y())

    print('\nConstraints')
    print('Demand  = ', model.demand())
    print('Labor A = ', model.laborA())
    print('Labor B = ', model.laborB())

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x + 30*y

3 Constraint Declarations
    demand : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :    x :  40.0 :   True
    laborA : Size=1, Index=None, Active=True
        Key  : Lower : Body  : Upper : Active
        None :  -Inf : x + y :  80.0 :   True
    laborB : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y : 100.0 :   True

6 Declarations: x y profit demand laborA laborB


## GLPK

[GLPK](https://en.wikibooks.org/wiki/GLPK) is a the open-source **G**NU **L**inear **P**rogramming **K**it available for use under the GNU General Public License 3. GLPK is a single-threaded simplex solver generally suited to small to medium scale linear-integer programming problems. It is written in C with minimal dependencies and therefore is highly portable among computers and operating systems

GLPK is often 'good enough' for many examples. For larger problem, however, users should consider higher-performance solvers, such as COIN-OR CLP, that take advantage of multi-threaded processors.

In [3]:
SolverFactory('glpk').solve(model, tee=True).write()

# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('Demand  = ', model.demand())
print('Labor A = ', model.laborA())
print('Labor B = ', model.laborB())

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmp5gh9fdaw.glpk.raw --wglp /tmp/tmpx0pp6dga.glpk.glp --cpxlp
 /tmp/tmps9u3tv5o.pyomo.lp
Reading problem data from '/tmp/tmps9u3tv5o.pyomo.lp'...
4 rows, 3 columns, 6 non-zeros
30 lines were read
Writing problem data to '/tmp/tmpx0pp6dga.glpk.glp'...
22 lines were written
GLPK Simplex Optimizer, v4.65
4 rows, 3 columns, 6 non-zeros
Preprocessing...
2 rows, 2 columns, 4 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.000e+00  ratio =  2.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 2
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (2)
*     3: obj =   2.600000000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40412 bytes)
Writing basic solution to '/tmp/tmp5gh9fdaw.glpk.raw'...
16 lines were written
# ==========================================================
# = Solver Results 

## COIN-OR Clp

[COIN-OR Clp](https://github.com/coin-or/Clp) is a multi-threaded open-source linear programming solver written in C++ and distributed under the Eclipse Public License (EPL). Clp is generally a good choice for linear programs that do not include any binary or integer variables. 

In [4]:
SolverFactory('clp').solve(model, tee=True).write()

display_solution(model)

Coin LP version 1.17.4, build Nov  3 2022
command line - /root/.idaes/bin/clp /tmp/tmpzl39eu31.pyomo.nl -AMPL 
CLP 1.17.4: # ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: CLP 1.17.4 optimal, objective -2600; 2 iterations
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.01854109764099121
# ----------------------------------------------------------
#   Solution Informa

## COIN-OR Cbc

[COIN-OR CBC](https://github.com/coin-or/Cbc) is a multi-threaded open-source **C**oin-or **b**ranch and **c**ut mixed-integer linear programming solver written in C++ under the Eclipse Public License (EPL). CBC is generally a good choice for a general purpose MILP solver for medium to large scale problems.

In [5]:
SolverFactory('cbc').solve(model, tee=True).write()

display_solution(model)

Welcome to the CBC MILP Solver 
Version: 2.10.4 
Build Date: Nov  3 2022 

command line - /root/.idaes/bin/cbc -printingOptions all -import /tmp/tmpmdebou2l.pyomo.lp -stat=1 -solve -solu /tmp/tmpmdebou2l.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 2 (-2) rows, 2 (-1) columns and 4 (-2) elements
Statistics for presolved model


Problem has 2 rows, 2 columns (2 with objective) and 4 elements
Column breakdown:
1 of type 0.0->inf, 1 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
2 of type L other, 0 of type Range 0.0->1.0, 0 of typ

## COIN-OR Ipopt installation

[COIN-OR Ipopt](https://github.com/coin-or/Ipopt) is an open-source **I**nterior **P**oint **Opt**imizer for large-scale nonlinear optimization available under the Eclipse Public License (EPL). It is well-suited to solving nonlinear programming problems without integer or binary constraints.

In [6]:
SolverFactory('ipopt').solve(model, tee=True).write()

display_solution(model)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

## COIN-OR Bonmin installation

[COIN-OR Bonmin](https://www.coin-or.org/Bonmin/Intro.html) is a **b**asic **o**pen-source solver for **n**onlinear **m**ixed-**in**teger programming problems (MINLP). It utilizes CBC and Ipopt for solving relaxed subproblems.

In [7]:
SolverFactory('bonmin').solve(model, tee=True).write()

display_solution(model)

Bonmin 1.8.8 using Cbc 2.10.4 and Ipopt 3.13.2
bonmin: 
Cbc3007W No integer variables - nothing to do

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a 

## COIN-OR Couenne installation

[COIN-OR Couenne](https://www.coin-or.org/Couenne/)  is attempts to find global optima for mixed-integer nonlinear programming problems (MINLP).

In [8]:
SolverFactory('couenne').solve(model, tee=True).write()

display_solution(model)

Couenne 0.5.8 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions: http://www.coin-or.org/Couenne
couenne: 
ANALYSIS TEST: Couenne: new cutoff value -2.6000000020e+03 (0.009072 seconds)
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -2600        6 0.002238
Couenne: new cutoff value -2.6000000260e+03 (0.011897 seconds)
Loaded instance "/tmp/tmpit3p06db.pyomo.nl"
Constraints:            3
Variables:              2 (0 integer)
Auxiliaries:            2 (0 integer)

Coin0506I Presolve 0 (-3) rows, 0 (-4) columns and 0 (-8) elements
Clp0000I Optimal - objective value -2600
Clp0032I Optimal objective -2600 - 0 iterations time 0.002, Presolve 0.00
Cbc3007W No integer variables - nothing to do
Clp0000I Optimal - objective value -2600

 	"Finished"

Linearization cuts added at root node:          3
Linearization cuts added in total:  